In [1]:
import numpy as np
import sys
import os
import pandas as pd
import plotly.graph_objects as go
from scipy.interpolate import griddata

sys.path.append(os.path.abspath("../libs"))
sys.path.append(os.path.abspath("../utils"))

from plots.plots_tarefa5 import plot_interpolated_surface_with_original_data
from levenberg_marquadt import levenberg_marquadt
from normalize import MinMaxNormalizer
from activations_fn import tanh_derivative
from network import (
    hidden_forward,
    make_jacobian_fn,
    make_residuals_fn,
    make_mse_loss_for_network,
    unflatten_weights,
)

pd.set_option("display.float_format", "{:.5f}".format)

# Ajuste de curva por otimização

## Carregar os dados

In [2]:
# Carregamento dos dados
df = pd.read_excel("../data/Trabalho5dados.xlsx")
df.columns = ["x1", "x2", "y"]

x1_data, x2_data = df["x1"], df["x2"]
y_values = df["y"].values
df.head()

,x1,x2,y
0,17.50000,576.20000,848
1,35.90000,598.60000,905
2,31.40000,612.10000,578
3,23.70000,624.20000,382
4,20.20000,635.20000,298


### Mostrar superfície dos dados originais

In [3]:
grid_size = 100
fig_surface = plot_interpolated_surface_with_original_data(x1_data, x2_data, y_values, grid_size=grid_size)

## Calcular as funções de perda

### Pré-processamento dos dados

In [ ]:
features = df[["x1", "x2"]]
y = df["y"]

# Criar os objetos para Padronização
scaler = MinMaxNormalizer(-1, 1)
scaler_y = MinMaxNormalizer(-1, 1)

# Cria as cópias dos dados para padronizar
scaled_features = features.copy()
scaled_y = y.copy()

# Ajusta os padronizadores aos dados
scaler.fit(scaled_features)
scaler_y.fit(scaled_y.to_frame())

# Padroniza os dados
scaled_features = scaler.normalize(scaled_features)
scaled_y = scaler_y.normalize(scaled_y.to_frame()).squeeze()

# Preparar dados
x1_scaled = scaled_features["x1"].values
x2_scaled = scaled_features["x2"].values
y_scaled = scaled_y.values

# Definir parâmetros da rede neural
n_neurons = 2
n_iterations = 10000
tolerance = 1e-6
alpha = 1e-3

# Inicializar pesos para cada neurônio (2 features + 1 bias)
hidden_weights = [np.random.randn(features.shape[1] + 1) for _ in range(n_neurons)]

# Neurônio de saída tem n_neurons + 1 pesos (saídas dos neurônios ocultos + bias)
output_weights = np.random.randn(n_neurons + 1)

# Combinar todos os pesos em uma lista
initial_weights_list = hidden_weights + [output_weights]

# Concatenar todos os pesos em um único vetor para o Levenberg-Marquardt
initial_weights_flat = np.concatenate([w.flatten() for w in initial_weights_list])

### Inicializar as funções de perda e gradiente

In [5]:
# Função de custo e gradiente
loss_function = make_mse_loss_for_network(
    x1_scaled, x2_scaled, y_scaled, activation_fn=np.tanh, n_neurons=n_neurons
)

# Função de resíduos
residuals_fn = make_residuals_fn(
    x1_scaled, x2_scaled, y_scaled, n_neurons=n_neurons, activation_fn=np.tanh
)

# Função da jacobiana
jacobian_fn = make_jacobian_fn(
    x1_scaled,
    x2_scaled,
    n_neurons=n_neurons,
    activation_fn=np.tanh,
    activation_deriv=tanh_derivative,
)

### Rodar os experimentos

In [6]:
print(f"Treinando rede com {n_neurons} neurônios na camada oculta...")

# Treinar com Levenberg-Marquardt
weights_flat, losses, n_iters = levenberg_marquadt(
    initial_weights_flat,
    residuals_fn,
    loss_function,
    jacobian_fn,
    alpha=alpha,
    alpha_variability=10,
    max_iter=n_iterations,
    tolerance=tolerance,
    stopping_criteria=[1, 3],
)

# Usar os pesos finais
neurons_weights_flat = weights_flat[-1]
neurons_weights = unflatten_weights(
    neurons_weights_flat, n_inputs=2, n_neurons=n_neurons
)

# Fazer previsão usando a rede treinada
y_hat_scaled = hidden_forward(
    x1_scaled, x2_scaled, neurons_weights=neurons_weights, activation_fn=np.tanh
)

# Convertendo para DataFrame com o mesmo índice usado no treinamento
y_hat = scaler_y.denormalize(y_hat_scaled.reshape(-1, 1)).flatten()

print("Treinamento concluído.")
print(f"Total de iterações: {n_iters}")
print(f"Perda final (MSE): {losses[-1]:.6f}")

Treinando rede com 2 neurônios na camada oculta...
Treinamento concluído.
Total de iterações: 17
Perda final (MSE): 0.024336


## Resultados

In [7]:
# Calcular métricas
mse_final = np.mean((y - y_hat) ** 2)
rmse_final = np.sqrt(mse_final)
mae_final = np.mean(np.abs(y - y_hat))

# Preparar resultados para exibição
dict_results = {
    "Feature_Set": "MinMax(-1,1)",
    "Loss_Function": "MSE",
    "Final_Loss": losses[-1],
    "MSE_Final": mse_final,
    "RMSE_Final": rmse_final,
    "MAE_Final": mae_final,
    "Iterations": n_iters,
}

# Adiciona os pesos de cada neurônio em colunas separadas
for i, weights in enumerate(neurons_weights[:-1]):  # Exclui o neurônio de saída
    dict_results[f"Pesos_Neuronio_{i + 1}"] = [float(f"{w:.6f}") for w in weights]

# Adiciona os pesos do neurônio de saída
dict_results["Pesos_Neuronio_Saida"] = [
    float(f"{w:.6f}") for w in neurons_weights[-1]
]

df_result = pd.DataFrame([dict_results])
df_result

,Feature_Set,Loss_Function,Final_Loss,MSE_Final,RMSE_Final,MAE_Final,Iterations,Pesos_Neuronio_1,Pesos_Neuronio_2,Pesos_Neuronio_Saida
0,"MinMax(-1,1)",MSE,0.02434,19147.06795,138.37293,102.71997,17,"[-3.075253, -3.712325, 4.080444]","[1.827193, -0.157874, 1.579975]","[0.373472, 1.765725, -1.425692]"


In [8]:
# Criar grade para visualizar a predição
x1_grid = np.linspace(min(x1_data), max(x1_data), grid_size)
x2_grid = np.linspace(min(x2_data), max(x2_data), grid_size)
x1_mesh, x2_mesh = np.meshgrid(x1_grid, x2_grid)
points = np.column_stack((x1_data, x2_data))

y_hat_grid = griddata(points, y_hat, (x1_mesh, x2_mesh), method="cubic")

""" # Adicionar superfície prevista
fig_surface.add_trace(go.Surface(
    x=x1_grid,
    y=x2_grid,
    z=y_hat_grid,
    colorscale='Reds',
    opacity=0.7,
    showscale=True,
    name='Superfície Prevista'
)) """

# Adicionar os pontos previstos
fig_surface.add_trace(
    go.Scatter3d(
        x=x1_data,
        y=x2_data,
        z=y_hat,
        mode="markers",
        marker=dict(
            size=4,
            color="blue",
        ),
        name="Predições",
    )
)

fig_surface.update_layout(
    title="Comparação entre Dados Originais e Previstos",
)

fig_surface.show()